In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [2]:
print(tf.__version__)

2.0.0-beta0


W0820 00:26:51.272106 15532 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [42]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
#Same number as integers
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#Number of test samples in similar format
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale )
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#Batching is only necessary for training because it provides average values (less accuracy),
#which is fine because we do backpropogating here. It also saves computational power.
#For testing and validation we use less computational power and nees maximum accurace. We forward propogate only. So no batching. 
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
#For validation we need a batch format too, but we don't want to create batches so we just create one per sample.
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


W0820 01:10:35.222804 15532 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [43]:
input_size = 784
output_size = 10
hidden_layer_size = 500

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax'),
    
                            ])

In [44]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [45]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 17s - loss: 0.2194 - accuracy: 0.9338 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 14s - loss: 0.0820 - accuracy: 0.9747 - val_loss: 0.0700 - val_accuracy: 0.9773
Epoch 3/5
540/540 - 15s - loss: 0.0560 - accuracy: 0.9829 - val_loss: 0.0550 - val_accuracy: 0.9832
Epoch 4/5
540/540 - 16s - loss: 0.0405 - accuracy: 0.9873 - val_loss: 0.0378 - val_accuracy: 0.9887
Epoch 5/5
540/540 - 20s - loss: 0.0310 - accuracy: 0.9906 - val_loss: 0.0336 - val_accuracy: 0.9895


In [47]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0738 - accuracy: 0.97 - 2s 2s/step - loss: 0.0738 - accuracy: 0.9787

In [48]:
print(test_loss, test_accuracy)

0.07376251369714737 0.9787
